In [15]:

#core enviroment libraries for RL 
import gym
from gym.spaces import Discrete, MultiDiscrete,Box, Dict, MultiBinary,Tuple

#utilities 
import numpy as np
import random
#these libraries have to do with the agents 
import ray
from ray.rllib.utils.deprecation import Deprecated
from ray.rllib.utils.metrics import (
    LAST_TARGET_UPDATE_TS,
    NUM_AGENT_STEPS_SAMPLED,
    NUM_ENV_STEPS_SAMPLED,
    NUM_TARGET_UPDATES,
    SYNCH_WORKER_WEIGHTS_TIMER,
)
from ray.rllib.utils.replay_buffers.utils import sample_min_n_steps_from_buffer
from ray.rllib.utils.typing import ResultDict, AlgorithmConfigDict
from ray.rllib.utils.deprecation import DEPRECATED_VALUE
from ray.rllib.utils.deprecation import deprecation_warning
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray import air, tune

import argparse
import logging
import os
#import pathpy as pp

from ray.tune import register_env
from ray.rllib.algorithms.qmix import QMixConfig
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved

import mne# preprocessing and brain importation and utilities library including acessing and preprocessing the EEG data
#these libraries have to do with the free energy principle
#import pymdp
#from pymdp import utils
#from pymdp.agent import Agent
#from gym.spaces import 

#optimization of deep learning and RL aspects of algorithm these will allow the algorithm to run faster with less memory 
#from composer import Trainer
#from nebullvm.api.functions import optimize_model 
from numba import jit

import taichi as ti
import taichi.math as tm
"""
dependency network

Qmix.py - has qmixpolicy.py as a dependency 
Qmixpolicy.py has  mixers.py and Model.py dependencies
Model.py -base
mixers.py -base

if we do this carefully we can use taichi to carry out speedup

The mixer and the bmodel would be ti.funcs

qmixpolicy would also be a ti.func

qmix would be the ti.kernel

"""

import tensorflow as tf

import ivy# library for interoperable across all deep learning frameworks 
#import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import torch
from laplace import Laplace #for model selection 
from laplace.baselaplace import FullLaplace
from laplace.curvature.backpack import BackPackGGN
#from nebulgym.decorators.torch_decorators import accel

#from nebulgym.decorators.torch_decorators import accelerate_model, accelerate_dataset

#below libraries are core libraries for q-mix Rllib algorithm
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override
from torch import nn

from ray.rllib.policy.torch_policy import TorchPolicy

from typing import Optional, Type,  Dict, List, Tuple

from ray.rllib.algorithms.simple_q.simple_q import SimpleQ, SimpleQConfig
from ray.rllib.algorithms.qmix.qmix_policy import QMixTorchPolicy
from ray.rllib.utils.replay_buffers.utils import update_priorities_in_replay_buffer
from ray.rllib.execution.rollout_ops import (
    synchronous_parallel_sample,
)

from ray.rllib.execution.train_ops import (
    multi_gpu_train_one_step,
    train_one_step,
)
from ray.rllib.policy.policy import Policy
from ray.rllib.models.preprocessors import get_preprocessor
import tree  # pip install dm_tree

from ray.rllib.algorithms.qmix.mixers import VDNMixer, QMixer
#from ray.rllib.algorithms.qmix.model import RNNModel, _get_size
from ray.rllib.env.multi_agent_env import MultiAgentEnv, ENV_STATE, make_multi_agent
from ray.rllib.env.wrappers.group_agents_wrapper import GROUP_REWARDS
from ray.rllib.models.catalog import ModelCatalog
from ray.rllib.models.modelv2 import _unpack_obs
from ray.rllib.models.torch.torch_action_dist import TorchCategorical
from ray.rllib.policy.rnn_sequencing import chop_into_sequences
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.metrics.learner_info import LEARNER_STATS_KEY
from ray.rllib.utils.typing import TensorType

from ray.tune.registry import get_trainable_cls

import pyro
import pyro.distributions as dist
from pyro.contrib.oed.eig import posterior_eig
from pyro.optim import Adam

import nitime
from deeptime.sindy import SINDy

#data visualization
import pygwalker as pyg

ti.init(arch=ti.gpu)

[Taichi] Starting on arch=cuda


In [16]:

def pmodel(design):

    # This line allows batching of designs, treating all batch dimensions as independent
    with pyro.plate_stack("plate_stack", design.shape):

        # We use a dirchlet prior for theta
        theta = pyro.sample("theta", dist.Normal(torch.tensor(0.0), torch.tensor(1.0)))
        #theta = pyro.sample("theta", dist.Dirichlet())
        # We use a simple logistic regression model for the likelihood
        logit_p = theta - design
        y = pyro.sample("y", dist.Bernoulli(logits=logit_p))

        return y

eig = nmc_eig(pmodel, design, observation_labels=["y"], target_labels=["theta"], N=2500, M=50)

NameError: name 'nmc_eig' is not defined

In [ ]:
import pyro
import pyro.distributions as dist

def pmodel(polling_allocation):
    # This allows us to run many copies of the model in parallel
    with pyro.plate_stack("plate_stack", polling_allocation.shape[:-1]):
        # Begin by sampling alpha
        alpha = pyro.sample("alpha", dist.MultivariateNormal(
            prior_mean, covariance_matrix=prior_covariance))

        # Sample y conditional on alpha
        poll_results = pyro.sample("y", dist.Binomial(
            polling_allocation, logits=alpha).to_event(1))

        # Now compute w according to the (approximate) electoral college formula
        dem_win = election_winner(alpha)
        pyro.sample("w", dist.Delta(dem_win))

        return poll_results, dem_win, alpha

In [ ]:
from torch import nn

class OutcomePredictor(nn.Module):

    def __init__(self):
        super().__init__()
        self.h1 = nn.Linear(51, 64)
        self.h2 = nn.Linear(64, 64)
        self.h3 = nn.Linear(64, 1)

    def compute_dem_probability(self, y):
        z = nn.functional.relu(self.h1(y))
        z = nn.functional.relu(self.h2(z))
        return self.h3(z)

    def forward(self, y_dict, design, observation_labels, target_labels):

        pyro.module("posterior_guide", self)

        y = y_dict["y"]
        dem_prob = self.compute_dem_probability(y).squeeze()
        pyro.sample("w", dist.Bernoulli(logits=dem_prob))

In [ ]:
prior_entropy = dist.Bernoulli(prior_w_prob).entropy()

from collections import OrderedDict

poll_in_florida = torch.zeros(51)
poll_in_florida[9] = 1000

poll_in_dc = torch.zeros(51)
poll_in_dc[8] = 1000

uniform_poll = (1000 // 51) * torch.ones(51)

# The swing score measures how close the state is to 50/50
swing_score = 1. / (.5 - torch.tensor(prior_prob_dem.sort_values("State").values).squeeze()).abs()
swing_poll = 1000 * swing_score / swing_score.sum()
swing_poll = swing_poll.round()

poll_strategies = OrderedDict([("Florida", poll_in_florida),
                               ("DC", poll_in_dc),
                               ("Uniform", uniform_poll),
                               ("Swing", swing_poll)])

In [ ]:
from pyro.contrib.oed.eig import posterior_eig
from pyro.optim import Adam

eigs = {}
best_strategy, best_eig = None, 0

for strategy, allocation in poll_strategies.items():
    print(strategy, end=" ")
    guide = OutcomePredictor()
    pyro.clear_param_store()
    # To reduce noise when comparing designs, we will use the precomputed value of H(p(w))
    # By passing eig=False, we tell Pyro not to estimate the prior entropy on each run
    # The return value of `posterior_eig` is then -E_p(w,y)[log q(w|y)]
    ape = posterior_eig(pmodel, allocation, "y", "w", 10, 12500, guide,
                        Adam({"lr": 0.001}), eig=False, final_num_samples=10000)
    eigs[strategy] = prior_entropy - ape
    print(eigs[strategy].item())
    if eigs[strategy] > best_eig:
        best_strategy, best_eig = strategy, eigs[strategy]

In [ ]:
#create this like the outcome predictor. This is the expected free energy sub-module ESM
#this will essentially calculate the  
class ESM(nn.Module,ivy.Module):
     def __init__(self):
        super().__init__()
        tfk = tf.keras
        tfkl = tf.keras.layers
        input_shape=2
        tfpl = tfp.layers
        prior = tfp.distributions.Dirichlet(concentration)

        tfd = tfp.distributions
        encoded_size = 16
        #model = tf.keras.Sequential([
        u = tfkl.InputLayer(input_shape=input_shape),
        
        u = tf.keras.layers.LSTM(25,kernel_initializer='zeros',activation='tanh', dtype = x.dtype, use_bias=True)(u),
        u = tfp.layers.VariationalGaussianProcess(
                num_inducing_points=num_inducing_points, kernel_provider=RBFKernelFn(dtype=x.dtype) , event_shape=(1,),
                inducing_index_points_initializer=tf.compat.v1.constant_initializer(
                    np.linspace(0,x_range, num=1125,
                                dtype=x.dtype)[..., np.newaxis]), unconstrained_observation_noise_variance_initializer=(tf.compat.v1.constant_initializer(np.log(np.expm1(1.)).astype(x.dtype))),variational_inducing_observations_scale_initializer=(tf.compat.v1.constant_initializer(np.log(np.expm1(1.)).astype(x.dtype))), mean_fn=None,
                jitter=1e-06, convert_to_tensor_fn=tfp.distributions.Distribution.sample)(u)

  
    #in unconstrained thing replace astype with tf.dtype thing.    #tf.initializers.constant(-10.0)
    #])
    def compute_dem_probability(self, y):
        #fk.Model()
        o = tf.nn.relu(u)
        return o 
    def forward(self, y_dict, design, observation_labels, target_labels):

        pyro.module("posterior_guide", self)

        y = y_dict["y"]
        
        #bmodel = FullLaplace(bmodel,'regression',prior_precision=2)
        dem_prob = self.compute_dem_probability(y).squeeze()
        pyro.sample("w", dist.Bernoulli(logits=dem_prob))
    

In [3]:
#expected free energy

class emodel(TorchModelV2, nn.Module,ivy.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):#(self, obs_space, action_space, num_outputs, model_config):
        TorchModelV2.__init__(
             self, obs_space, action_space, num_outputs, model_config, name#self, obs_space, action_space, num_outputs, model_config
        )
        nn.Module.__init__(self)
        self.obs_size = _get_size(obs_space)
        self.rnn_hidden_dim = model_config["lstm_cell_size"]

        
        """
        pseudo code
        
        assign a prior to be the posterior of the output of previous agent
        
        if no output of agent is found we have a preset prior. Prior is going to be a dirchlet
        
        """
        
        guide = ESM()
        pyro.clear_param_store()
        # To reduce noise when comparing designs, we will use the precomputed value of H(p(w))
        # By passing eig=False, we tell Pyro not to estimate the prior entropy on each run
        # The return value of `posterior_eig` is then -E_p(w,y)[log q(w|y)]
        
        #ape = average posterior entropy
        ape = posterior_eig(model, allocation, "y", "w", 10, 12500, guide,
                            Adam({"lr": 0.001}), eig=False, final_num_samples=10000)
        eigs[strategy] = prior_entropy - ape
        print(eigs[strategy].item())
        if eigs[strategy] > best_eig:
            best_strategy, best_eig = strategy, eigs[strategy]
            
        pragmatic_value = 0
        

        """
        Next objective: determine the probaility of hidden state given observation
        
        we should use a neural network to calculate the probability of reward given action 
        
        if there is 0 correlation, that is to say reward does not change given a certain action then p(s|y) = 0
        
        this is the pragmatic value
        
        """
        self.fc1 = nn.Linear(self.obs_size, self.rnn_hidden_dim)
        self.rnn = nn.GRUCell(self.rnn_hidden_dim, self.rnn_hidden_dim)
        
        self.fc2 = nn.Linear(self.rnn_hidden_dim, num_outputs)
        self.n_agents = model_config["n_agents"]

    @override(ModelV2)
    def get_initial_state(self):

        return [
            self.fc1.weight.new(self.n_agents, self.rnn_hidden_dim).zero_().squeeze(0)
        ]

    @override(ModelV2)
    def forward(self,x,input_dict, hidden_state, seq_lens):

        #negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
                # Place hidden states on same device as model.
        
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        h_in = hidden_state[0].reshape(-1, self.rnn_hidden_dim)
        h = self.rnn(x, h_in)
        #q = self.fc2(h)
        vae = tfk.Model(inputs=input_dict["obs_flat"],
                outputs=decoder(input_dict["obs_flat"][0]))
        return q, [h]

def _get_size(obs_space):
    return get_preprocessor(obs_space)(obs_space).size
ivy.set_framework('torch')

tf.Tensor(1.504077, shape=(), dtype=float32)


In [17]:
torch, nn = try_import_torch()
#@ti.func
class bmodel(TorchModelV2, nn.Module,ivy.Module):
    """The default RNN model for QMIX."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):#(self, obs_space, action_space, num_outputs, model_config):
        TorchModelV2.__init__(
             self, obs_space, action_space, num_outputs, model_config, name#self, obs_space, action_space, num_outputs, model_config
        )
        nn.Module.__init__(self)
        self.obs_size = _get_size(obs_space)
        self.rnn_hidden_dim = model_config["lstm_cell_size"]
        tfk = tf.keras
        tfkl = tf.keras.layers
        input_shape=2
        tfpl = tfp.layers
        tfd = tfp.distributions

        tfd = tfp.distributions
        encoded_size = 16
        
        """
        pseudo code
        
        assign a prior to be the posterior of the output of previous agent
        
        if no output of agent is found we have a preset prior. Prior is going to be a dirchlet
        
        """
        prior = tfp.distributions.Dirichlet(concentration)
        prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                            reinterpreted_batch_ndims=1)
        tfpl = tfp.layers
        encoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=input_shape),
            #tfkl.Dense(8)
            tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
                          activation=None),

            tfpl.MultivariateNormalTriL(
                    encoded_size,
                    activity_regularizer=tfpl.KLDivergenceRegularizer(priora, weight=1.2)),
        ])
        decoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=[encoded_size]),
            tfkl.Dense(tfpl.IndependentNormal.params_size(encoded_size))
            #tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits)

        ])
        self.fc1 = nn.Linear(self.obs_size, self.rnn_hidden_dim)
        self.rnn = nn.GRUCell(self.rnn_hidden_dim, self.rnn_hidden_dim)
        self.fc2 = nn.Linear(self.rnn_hidden_dim, num_outputs)
        self.n_agents = model_config["n_agents"]

    @override(ModelV2)
    def get_initial_state(self):

        return [
            self.fc1.weight.new(self.n_agents, self.rnn_hidden_dim).zero_().squeeze(0)
        ]

    @override(ModelV2)
    def forward(self,x,input_dict, hidden_state, seq_lens):

        #negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
                # Place hidden states on same device as model.
        
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        h_in = hidden_state[0].reshape(-1, self.rnn_hidden_dim)
        h = self.rnn(x, h_in)
        #q = self.fc2(h)
        vae = tfk.Model(inputs=input_dict["obs_flat"],
                outputs=decoder(input_dict["obs_flat"][0]))
        kl = tf.keras.losses.KLDivergence()
        model.add_loss(kl)
        return q, [h]

def _get_size(obs_space):
    return get_preprocessor(obs_space)(obs_space).size

In [ ]:
torch, nn = try_import_torch()
#@ti.func
class bmodel(TorchModelV2, nn.Module,ivy.Module):
    """The default RNN model for QMIX."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):#(self, obs_space, action_space, num_outputs, model_config):
        TorchModelV2.__init__(
             self, obs_space, action_space, num_outputs, model_config, name#self, obs_space, action_space, num_outputs, model_config
        )
        nn.Module.__init__(self)
        self.obs_size = _get_size(obs_space)
        self.rnn_hidden_dim = model_config["lstm_cell_size"]
        tfk = tf.keras
        tfkl = tf.keras.layers
        input_shape=2
        tfpl = tfp.layers
        tfd = tfp.distributions

        tfd = tfp.distributions
        encoded_size = 16
        
        """
        pseudo code
        
        assign a prior to be the posterior of the output of previous agent
        
        if no output of agent is found we have a preset prior. Prior is going to be a dirchlet
        
        """
        prior = tfp.distributions.Dirichlet(concentration)
        prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                            reinterpreted_batch_ndims=1)
        tfpl = tfp.layers
        encoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=input_shape),
            #tfkl.Dense(8)
            tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
                          activation=None),

            tfpl.MultivariateNormalTriL(
                    encoded_size,
                    activity_regularizer=tfpl.KLDivergenceRegularizer(priora, weight=1.2)),
        ])
        decoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=[encoded_size]),
            tfkl.Dense(tfpl.IndependentNormal.params_size(encoded_size))
            #tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits)

        ])
        self.fc1 = nn.Linear(self.obs_size, self.rnn_hidden_dim)
        self.rnn = nn.GRUCell(self.rnn_hidden_dim, self.rnn_hidden_dim)
        self.fc2 = nn.Linear(self.rnn_hidden_dim, num_outputs)
        self.n_agents = model_config["n_agents"]

    @override(ModelV2)
    def get_initial_state(self):

        return [
            self.fc1.weight.new(self.n_agents, self.rnn_hidden_dim).zero_().squeeze(0)
        ]

    @override(ModelV2)
    def forward(self,x,input_dict, hidden_state, seq_lens):

        #negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
                # Place hidden states on same device as model.
        
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        h_in = hidden_state[0].reshape(-1, self.rnn_hidden_dim)
        h = self.rnn(x, h_in)
        #q = self.fc2(h)
        vae = tfk.Model(inputs=input_dict["obs_flat"],
                outputs=decoder(input_dict["obs_flat"][0]))
        kl = tf.keras.losses.KLDivergence()
        model.add_loss(kl)
        return q, [h]

def _get_size(obs_space):
    return get_preprocessor(obs_space)(obs_space).size

In [20]:
import pyro
#guide = pyro.infer.autoguide.AutoNormal(model)

#elbo = elbo_(model, guide)

In [18]:
from ray.rllib.utils.framework import try_import_torch

torch, nn = try_import_torch()

#@ti.func
class QMixer(nn.Module):
    def __init__(self, n_agents, state_shape, mixing_embed_dim):
        super(QMixer, self).__init__()

        self.n_agents = n_agents
        self.embed_dim = mixing_embed_dim
        self.state_dim = int(np.prod(state_shape))

        self.hyper_w_1 = nn.Linear(self.state_dim, self.embed_dim * self.n_agents)
        self.hyper_w_final = nn.Linear(self.state_dim, self.embed_dim)

        # State dependent bias for hidden layer
        self.hyper_b_1 = nn.Linear(self.state_dim, self.embed_dim)

        # V(s) instead of a bias for the last layers
        self.V = nn.Sequential(
            nn.Linear(self.state_dim, self.embed_dim),
            nn.ReLU(),
            nn.Linear(self.embed_dim, 1),
        )
        
    def forward(self, agent_qs, states):
        """Forward pass for the mixer.
        Args:
            agent_qs: Tensor of shape [B, T, n_agents, n_actions]
            states: Tensor of shape [B, T, state_dim]
        """
        bs = agent_qs.size(0)
        states = states.reshape(-1, self.state_dim)
        agent_qs = agent_qs.view(-1, 1, self.n_agents)
        # First layer
        w1 = torch.abs(self.hyper_w_1(states))
        b1 = self.hyper_b_1(states)
        w1 = w1.view(-1, self.n_agents, self.embed_dim)
        b1 = b1.view(-1, 1, self.embed_dim)
        hidden = nn.functional.elu(torch.bmm(agent_qs, w1) + b1)
        # Second layer
        w_final = torch.abs(self.hyper_w_final(states))
        w_final = w_final.view(-1, self.embed_dim, 1)
        # State-dependent bias
        v = self.V(states).view(-1, 1, 1)
        # Compute final output
        y = torch.bmm(hidden, w_final) + v
        # Reshape and return
        q_tot = y.view(bs, -1, 1)
        return q_tot

In [22]:
#@ti.func
class QMixLoss(nn.Module):
    def __init__(
        self,
        model,
        target_model,
        mixer,
        target_mixer,
        n_agents,
        n_actions,
        double_q=True,
        gamma=0.99,
    ):
        nn.Module.__init__(self)
        self.model = model
        self.target_model = target_model
        self.mixer = mixer
        self.target_mixer = target_mixer
        self.n_agents = n_agents
        self.n_actions = n_actions
        self.double_q = double_q
        self.gamma = gamma

    def forward(
        self,
        rewards,
        actions,
        terminated,
        mask,
        obs,
        next_obs,
        action_mask,
        next_action_mask,
        state=None,
        next_state=None,
    ):
        """Forward pass of the loss.
        Args:
            rewards: Tensor of shape [B, T, n_agents]
            actions: Tensor of shape [B, T, n_agents]
            terminated: Tensor of shape [B, T, n_agents]
            mask: Tensor of shape [B, T, n_agents]
            obs: Tensor of shape [B, T, n_agents, obs_size]
            next_obs: Tensor of shape [B, T, n_agents, obs_size]
            action_mask: Tensor of shape [B, T, n_agents, n_actions]
            next_action_mask: Tensor of shape [B, T, n_agents, n_actions]
            state: Tensor of shape [B, T, state_dim] (optional)
            next_state: Tensor of shape [B, T, state_dim] (optional)
        """

        # Assert either none or both of state and next_state are given
        if state is None and next_state is None:
            state = obs  # default to state being all agents' observations
            next_state = next_obs
        elif (state is None) != (next_state is None):
            raise ValueError(
                "Expected either neither or both of `state` and "
                "`next_state` to be given. Got: "
                "\n`state` = {}\n`next_state` = {}".format(state, next_state)
            )

        # Calculate estimated Q-Values
        mac_out = _unroll_mac(self.model, obs)

        # Pick the Q-Values for the actions taken -> [B * n_agents, T]
        chosen_action_qvals = torch.gather(
            mac_out, dim=3, index=actions.unsqueeze(3)
        ).squeeze(3)

        # Calculate the Q-Values necessary for the target
        target_mac_out = _unroll_mac(self.target_model, next_obs)

        # Mask out unavailable actions for the t+1 step
        ignore_action_tp1 = (next_action_mask == 0) & (mask == 1).unsqueeze(-1)
        target_mac_out[ignore_action_tp1] = -np.inf

        # Max over target Q-Values
        if self.double_q:
            # Double Q learning computes the target Q values by selecting the
            # t+1 timestep action according to the "policy" neural network and
            # then estimating the Q-value of that action with the "target"
            # neural network
            
            #target neural network does expected free energy while policy
            #neural network will be variational free energy

            # Compute the t+1 Q-values to be used in action selection
            # using next_obs
            mac_out_tp1 = _unroll_mac(self.model, next_obs)

            # mask out unallowed actions
            mac_out_tp1[ignore_action_tp1] = -np.inf

            # obtain best actions at t+1 according to policy NN
            cur_max_actions = mac_out_tp1.argmax(dim=3, keepdim=True)

            # use the target network to estimate the Q-values of policy
            # network's selected actions
            target_max_qvals = torch.gather(target_mac_out, 3, cur_max_actions).squeeze(
                3
            )
        else:
            target_max_qvals = target_mac_out.max(dim=3)[0]

        assert (
            target_max_qvals.min().item() != -np.inf
        ), "target_max_qvals contains a masked action; \
            there may be a state with no valid actions."

        # Mix
        if self.mixer is not None:
            chosen_action_qvals = self.mixer(chosen_action_qvals, state)
            target_max_qvals = self.target_mixer(target_max_qvals, next_state)

        # Calculate 1-step Q-Learning targets
        targets = rewards + self.gamma * (1 - terminated) * target_max_qvals
        """
        
        guide = pyro.infer.autoguide.AutoNormal(model)

        elbo = elbo_(model, guide)
        
        el = elbo(data)
        
        #rewards = {self.agent_1:kullbacker, self.agent_2: el}
        #loss = lambda y, rv_y: rv_y.variational_loss(y, kl_weight=np.array(batch_size, x.dtype) / x.shape[0])
                
        
        """
        
        
        # Td-error
        #we need to replace this with a variational free energy error
        td_error = chosen_action_qvals - targets.detach()
        #te_error= tf.keras.losses.KLDivergence(chosen_action_qvals - targets.detach()).numpy()
        #negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)#this is ELBO 
        
        #we are going to ahve a kldivergence loss and a -ELBO regularizer for the mixer network 
        
        mask = mask.expand_as(td_error)

        # 0-out the targets that came from padded data
        masked_td_error = td_error * mask

        # Normal L2 loss, take mean over actual data
        
        #guide = pyro.infer.autoguide.AutoNormal(model)

        #elbo_ = pyro.infer.Trace_ELBO(num_particles=1)

        # Fix the model/guide pair
        #elbo = elbo_(model, guide)        
        
        #data = obs + preds 
        #loss = -ln()
        #loss = lambda y, rv_y: -rv_y.log_prob(y)
        loss = (masked_td_error**2).sum() / mask.sum()
        return loss, mask, masked_td_error, chosen_action_qvals, targets

    
#this part just above is what we need to revise
    
#@ti.func
class QMixTorchPolicy(TorchPolicy):
    """QMix impl. Assumes homogeneous agents for now.
    You must use MultiAgentEnv.with_agent_groups() to group agents
    together for QMix. This creates the proper Tuple obs/action spaces and
    populates the '_group_rewards' info field.
    Action masking: to specify an action mask for individual agents, use a
    dict space with an action_mask key, e.g. {"obs": ob, "action_mask": mask}.
    The mask space must be `Box(0, 1, (n_actions,))`.
    """

    def __init__(self, obs_space, action_space, config):
        # We want to error out on instantiation and not on import, because tune
        # imports all RLlib algorithms when registering them
        # TODO (Artur): Find a way to only import algorithms when needed
        if not torch:
            raise ImportError("Could not import PyTorch, which QMix requires.")

        _validate(obs_space, action_space)
        config = dict(ray.rllib.algorithms.qmix.qmix.DEFAULT_CONFIG, **config)
        self.framework = "torch"

        self.n_agents = len(obs_space.original_space.spaces)
        config["model"]["n_agents"] = self.n_agents
        self.n_actions = action_space.spaces[0].n
        self.h_size = config["model"]["lstm_cell_size"]
        self.has_env_global_state = False
        self.has_action_mask = False

        agent_obs_space = obs_space.original_space.spaces[0]
        if isinstance(agent_obs_space, gym.spaces.Dict):
            space_keys = set(agent_obs_space.spaces.keys())
            if "obs" not in space_keys:
                raise ValueError("Dict obs space must have subspace labeled `obs`")
            self.obs_size = _get_size(agent_obs_space.spaces["obs"])
            if "action_mask" in space_keys:
                mask_shape = tuple(agent_obs_space.spaces["action_mask"].shape)
                if mask_shape != (self.n_actions,):
                    raise ValueError(
                        "Action mask shape must be {}, got {}".format(
                            (self.n_actions,), mask_shape
                        )
                    )
                self.has_action_mask = True
            if ENV_STATE in space_keys:
                self.env_global_state_shape = _get_size(
                    agent_obs_space.spaces[ENV_STATE]
                )
                self.has_env_global_state = True
            else:
                self.env_global_state_shape = (self.obs_size, self.n_agents)
            # The real agent obs space is nested inside the dict
            config["model"]["full_obs_space"] = agent_obs_space
            agent_obs_space = agent_obs_space.spaces["obs"]
        else:
            self.obs_size = _get_size(agent_obs_space)
            self.env_global_state_shape = (self.obs_size, self.n_agents)
        #model = bmodel()#CModel()#CModel()
        #bmodel = Laplace(model,'regression', subset_of_weights='last_layer', hessian_structure='full')#
        ivy.set_framework('torch')
        #model = Laplace(model,'regression', subset_of_weights='last_layer', hessian_structure='full')
        bmodel = bmodel()
        bmodel = FullLaplace(bmodel,'regression',prior_precision=2)#0.0000000000000000000001)
        self.model = ModelCatalog.get_model_v2(
            agent_obs_space,
            action_space.spaces[0],
            self.n_actions,
            config["model"],
            framework="torch",
            name="model",
            default_model=bmodel#a#RNNModel#bmodel()#RNNModel,
        )

        super().__init__(obs_space, action_space, config, model=self.model)

        self.target_model = ModelCatalog.get_model_v2(
            agent_obs_space,
            action_space.spaces[0],
            self.n_actions,
            config["model"],
            framework="torch",
            name="target_model",
            default_model=bmodel#bmodel()#RNNModel
        ).to(self.device)

        self.exploration = self._create_exploration()

        # Setup the mixer network.
        if config["mixer"] is None:
            self.mixer = None
            self.target_mixer = None
        elif config["mixer"] == "qmix":
            self.mixer = QMixer(
                self.n_agents, self.env_global_state_shape, config["mixing_embed_dim"]
            ).to(self.device)
            self.target_mixer = QMixer(
                self.n_agents, self.env_global_state_shape, config["mixing_embed_dim"]
            ).to(self.device)

        self.cur_epsilon = 1.0
        self.update_target()  # initial sync

        # Setup optimizer
        self.params = list(self.model.parameters())
        if self.mixer:
            self.params += list(self.mixer.parameters())
        self.loss = QMixLoss(
            self.model,
            self.target_model,
            self.mixer,
            self.target_mixer,
            self.n_agents,
            self.n_actions,
            self.config["double_q"],
            self.config["gamma"],
        )
        from torch.optim import RMSprop

        self.rmsprop_optimizer = RMSprop(
            params=self.params,
            lr=config["lr"],
            alpha=config["optim_alpha"],
            eps=config["optim_eps"],
        )

    @override(TorchPolicy)
    def compute_actions_from_input_dict(
        self,
        input_dict: Dict[str, TensorType],
        explore: bool = None,
        timestep: Optional[int] = None,
        **kwargs,
    ) -> Tuple[TensorType, List[TensorType], Dict[str, TensorType]]:

        obs_batch = input_dict[SampleBatch.OBS]
        state_batches = []
        i = 0
        while f"state_in_{i}" in input_dict:
            state_batches.append(input_dict[f"state_in_{i}"])
            i += 1

        explore = explore if explore is not None else self.config["explore"]
        obs_batch, action_mask, _ = self._unpack_observation(obs_batch)
        # We need to ensure we do not use the env global state
        # to compute actions

        # Compute actions
        with torch.no_grad():
            q_values, hiddens = _mac(
                self.model,
                torch.as_tensor(obs_batch, dtype=torch.float, device=self.device),
                [
                    torch.as_tensor(np.array(s), dtype=torch.float, device=self.device)
                    for s in state_batches
                ],
            )
            avail = torch.as_tensor(action_mask, dtype=torch.float, device=self.device)
            masked_q_values = q_values.clone()
            masked_q_values[avail == 0.0] = -float("inf")
            masked_q_values_folded = torch.reshape(
                masked_q_values, [-1] + list(masked_q_values.shape)[2:]
            )
            actions, _ = self.exploration.get_exploration_action(
                action_distribution=TorchCategorical(masked_q_values_folded),
                timestep=timestep,
                explore=explore,
            )
            actions = (
                torch.reshape(actions, list(masked_q_values.shape)[:-1]).cpu().numpy()
            )
            hiddens = [s.cpu().numpy() for s in hiddens]

        return tuple(actions.transpose([1, 0])), hiddens, {}

    @override(TorchPolicy)
    def compute_actions(self, *args, **kwargs):
        return self.compute_actions_from_input_dict(*args, **kwargs)

    @override(TorchPolicy)
    def compute_log_likelihoods(
        self,
        actions,
        obs_batch,
        state_batches=None,
        prev_action_batch=None,
        prev_reward_batch=None,
    ):
        obs_batch, action_mask, _ = self._unpack_observation(obs_batch)
        return np.zeros(obs_batch.size()[0])

    @override(TorchPolicy)
    def learn_on_batch(self, samples):
        obs_batch, action_mask, env_global_state = self._unpack_observation(
            samples[SampleBatch.CUR_OBS]
        )
        (
            next_obs_batch,
            next_action_mask,
            next_env_global_state,
        ) = self._unpack_observation(samples[SampleBatch.NEXT_OBS])
        group_rewards = self._get_group_rewards(samples[SampleBatch.INFOS])

        input_list = [
            group_rewards,
            action_mask,
            next_action_mask,
            samples[SampleBatch.ACTIONS],
            samples[SampleBatch.TERMINATEDS],
            obs_batch,
            next_obs_batch,
        ]
        if self.has_env_global_state:
            input_list.extend([env_global_state, next_env_global_state])

        output_list, _, seq_lens = chop_into_sequences(
            episode_ids=samples[SampleBatch.EPS_ID],
            unroll_ids=samples[SampleBatch.UNROLL_ID],
            agent_indices=samples[SampleBatch.AGENT_INDEX],
            feature_columns=input_list,
            state_columns=[],  # RNN states not used here
            max_seq_len=self.config["model"]["max_seq_len"],
            dynamic_max=True,
        )
        # These will be padded to shape [B * T, ...]
        if self.has_env_global_state:
            (
                rew,
                action_mask,
                next_action_mask,
                act,
                terminateds,
                obs,
                next_obs,
                env_global_state,
                next_env_global_state,
            ) = output_list
        else:
            (
                rew,
                action_mask,
                next_action_mask,
                act,
                terminateds,
                obs,
                next_obs,
            ) = output_list
        B, T = len(seq_lens), max(seq_lens)

        def to_batches(arr, dtype):
            new_shape = [B, T] + list(arr.shape[1:])
            return torch.as_tensor(
                np.reshape(arr, new_shape), dtype=dtype, device=self.device
            )

        rewards = to_batches(rew, torch.float)
        actions = to_batches(act, torch.long)
        obs = to_batches(obs, torch.float).reshape([B, T, self.n_agents, self.obs_size])
        action_mask = to_batches(action_mask, torch.float)
        next_obs = to_batches(next_obs, torch.float).reshape(
            [B, T, self.n_agents, self.obs_size]
        )
        next_action_mask = to_batches(next_action_mask, torch.float)
        if self.has_env_global_state:
            env_global_state = to_batches(env_global_state, torch.float)
            next_env_global_state = to_batches(next_env_global_state, torch.float)

        # TODO(ekl) this treats group termination as individual termination
        terminated = (
            to_batches(terminateds, torch.float)
            .unsqueeze(2)
            .expand(B, T, self.n_agents)
        )

        # Create mask for where index is < unpadded sequence length
        filled = np.reshape(
            np.tile(np.arange(T, dtype=np.float32), B), [B, T]
        ) < np.expand_dims(seq_lens, 1)
        mask = (
            torch.as_tensor(filled, dtype=torch.float, device=self.device)
            .unsqueeze(2)
            .expand(B, T, self.n_agents)
        )

        # Compute loss
        loss_out, mask, masked_td_error, chosen_action_qvals, targets = self.loss(
            rewards,
            actions,
            terminated,
            mask,
            obs,
            next_obs,
            action_mask,
            next_action_mask,
            env_global_state,
            next_env_global_state,
        )

        # Optimise
        self.rmsprop_optimizer.zero_grad()

        loss_out.backward()
        grad_norm_info = apply_grad_clipping(self, self.rmsprop_optimizer, loss_out)
        self.rmsprop_optimizer.step()

        mask_elems = mask.sum().item()
        stats = {
            "loss": loss_out.item(),
            "td_error_abs": masked_td_error.abs().sum().item() / mask_elems,
            "q_taken_mean": (chosen_action_qvals * mask).sum().item() / mask_elems,
            "target_mean": (targets * mask).sum().item() / mask_elems,
        }
        stats.update(grad_norm_info)

        return {LEARNER_STATS_KEY: stats}

    @override(TorchPolicy)
    def get_initial_state(self):  # initial RNN state
        return [
            s.expand([self.n_agents, -1]).cpu().numpy()
            for s in self.model.get_initial_state()
        ]

    @override(TorchPolicy)
    def get_weights(self):
        return {
            "model": self._cpu_dict(self.model.state_dict()),
            "target_model": self._cpu_dict(self.target_model.state_dict()),
            "mixer": self._cpu_dict(self.mixer.state_dict()) if self.mixer else None,
            "target_mixer": self._cpu_dict(self.target_mixer.state_dict())
            if self.mixer
            else None,
        }

    @override(TorchPolicy)
    def set_weights(self, weights):
        self.model.load_state_dict(self._device_dict(weights["model"]))
        self.target_model.load_state_dict(self._device_dict(weights["target_model"]))
        if weights["mixer"] is not None:
            self.mixer.load_state_dict(self._device_dict(weights["mixer"]))
            self.target_mixer.load_state_dict(
                self._device_dict(weights["target_mixer"])
            )

    @override(TorchPolicy)
    def get_state(self):
        state = self.get_weights()
        state["cur_epsilon"] = self.cur_epsilon
        return state

    @override(TorchPolicy)
    def set_state(self, state):
        self.set_weights(state)
        self.set_epsilon(state["cur_epsilon"])

    def update_target(self):
        self.target_model.load_state_dict(self.model.state_dict())
        if self.mixer is not None:
            self.target_mixer.load_state_dict(self.mixer.state_dict())
        logger.debug("Updated target networks")

    def set_epsilon(self, epsilon):
        self.cur_epsilon = epsilon

    def _get_group_rewards(self, info_batch):
        group_rewards = np.array(
            [info.get(GROUP_REWARDS, [0.0] * self.n_agents) for info in info_batch]
        )
        return group_rewards

    def _device_dict(self, state_dict):
        return {
            k: torch.as_tensor(v, device=self.device) for k, v in state_dict.items()
        }

    @staticmethod
    def _cpu_dict(state_dict):
        return {k: v.cpu().detach().numpy() for k, v in state_dict.items()}

    def _unpack_observation(self, obs_batch):
        """Unpacks the observation, action mask, and state (if present)
        from agent grouping.
        Returns:
            obs (np.ndarray): obs tensor of shape [B, n_agents, obs_size]
            mask (np.ndarray): action mask, if any
            state (np.ndarray or None): state tensor of shape [B, state_size]
                or None if it is not in the batch
        """

        unpacked = _unpack_obs(
            np.array(obs_batch, dtype=np.float32),
            self.observation_space.original_space,
            tensorlib=np,
        )

        if isinstance(unpacked[0], dict):
            assert "obs" in unpacked[0]
            unpacked_obs = [np.concatenate(tree.flatten(u["obs"]), 1) for u in unpacked]
        else:
            unpacked_obs = unpacked

        obs = np.concatenate(unpacked_obs, axis=1).reshape(
            [len(obs_batch), self.n_agents, self.obs_size]
        )

        if self.has_action_mask:
            action_mask = np.concatenate(
                [o["action_mask"] for o in unpacked], axis=1
            ).reshape([len(obs_batch), self.n_agents, self.n_actions])
        else:
            action_mask = np.ones(
                [len(obs_batch), self.n_agents, self.n_actions], dtype=np.float32
            )

        if self.has_env_global_state:
            state = np.concatenate(tree.flatten(unpacked[0][ENV_STATE]), 1)
        else:
            state = None
        return obs, action_mask, state

#@ti.func
def _validate(obs_space, action_space):
    if not hasattr(obs_space, "original_space") or not isinstance(
        obs_space.original_space, gym.spaces.Tuple
    ):
        raise ValueError(
            "Obs space must be a Tuple, got {}. Use ".format(obs_space)
            + "MultiAgentEnv.with_agent_groups() to group related "
            "agents for QMix."
        )
    if not isinstance(action_space, gym.spaces.Tuple):
        raise ValueError(
            "Action space must be a Tuple, got {}. ".format(action_space)
            + "Use MultiAgentEnv.with_agent_groups() to group related "
            "agents for QMix."
        )
    if not isinstance(action_space.spaces[0], gym.spaces.Discrete):
        raise ValueError(
            "QMix requires a discrete action space, got {}".format(
                action_space.spaces[0]
            )
        )
    if len({str(x) for x in obs_space.original_space.spaces}) > 1:
        raise ValueError(
            "Implementation limitation: observations of grouped agents "
            "must be homogeneous, got {}".format(obs_space.original_space.spaces)
        )
    if len({str(x) for x in action_space.spaces}) > 1:
        raise ValueError(
            "Implementation limitation: action space of grouped agents "
            "must be homogeneous, got {}".format(action_space.spaces)
        )

#@ti.func
def _mac(model, obs, h):
    """Forward pass of the multi-agent controller.
    Args:
        model: TorchModelV2 class
        obs: Tensor of shape [B, n_agents, obs_size]
        h: List of tensors of shape [B, n_agents, h_size]
    Returns:
        q_vals: Tensor of shape [B, n_agents, n_actions]
        h: Tensor of shape [B, n_agents, h_size]
    """
    B, n_agents = obs.size(0), obs.size(1)
    if not isinstance(obs, dict):
        obs = {"obs": obs}
    obs_agents_as_batches = {k: _drop_agent_dim(v) for k, v in obs.items()}
    h_flat = [s.reshape([B * n_agents, -1]) for s in h]
    q_flat, h_flat = model(obs_agents_as_batches, h_flat, None)
    return q_flat.reshape([B, n_agents, -1]), [
        s.reshape([B, n_agents, -1]) for s in h_flat
    ]
#@ti.func
def _unroll_mac(model, obs_tensor):
    """Computes the estimated Q values for an entire trajectory batch"""
    B = obs_tensor.size(0)
    T = obs_tensor.size(1)
    n_agents = obs_tensor.size(2)

    mac_out = []
    h = [s.expand([B, n_agents, -1]) for s in model.get_initial_state()]
    for t in range(T):
        q, h = _mac(model, obs_tensor[:, t], h)
        mac_out.append(q)
    mac_out = torch.stack(mac_out, dim=1)  # Concat over time

    return mac_out
#@ti.func
def _drop_agent_dim(T):
    shape = list(T.shape)
    B, n_agents = shape[0], shape[1]
    return T.reshape([B * n_agents] + shape[2:])
#@ti.func
def _add_agent_dim(T, n_agents):
    shape = list(T.shape)
    B = shape[0] // n_agents
    assert shape[0] % n_agents == 0
    return T.reshape([B, n_agents] + shape[1:])

In [23]:
#@ti.func
#@ti.kernel
#@ti.data_oriented
class TwoStepGame(MultiAgentEnv):
    action_space = Discrete(2)

    def __init__(self, env_config):
        super().__init__()
        self.action_space = Discrete(2)
        self.state = None
        self.agent_1 = 0
        self.agent_2 = 1
        #self.agent_3=2
        self._skip_env_checking = True
        # MADDPG emits action logits instead of actual discrete actions
        self.actions_are_logits = env_config.get("actions_are_logits", False)
        self.one_hot_state_encoding = env_config.get("one_hot_state_encoding", False)
        self.with_state = env_config.get("separate_state_space", False)
        self._agent_ids = {0, 1}
        if not self.one_hot_state_encoding:
            self.observation_space = Discrete(6)
            self.with_state = False
        else:
            # Each agent gets the full state (one-hot encoding of which of the
            # three states are active) as input with the receiving agent's
            # ID (1 or 2) concatenated onto the end.
            if self.with_state:
                self.observation_space = Dict(
                    {
                        "obs": MultiDiscrete([2, 2, 2, 3]),
                        ENV_STATE: MultiDiscrete([2, 2, 2]),
                    }
                )
            else:
                self.observation_space = MultiDiscrete([2, 2, 2, 3])

    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        self.state = np.array([1, 0, 0])
        return self._obs(), {}

    def step(self, action_dict):

        state_index = np.flatnonzero(self.state)
        if state_index == 0:
            action = action_dict[self.agent_1]
            assert action in [0, 1], action
            if action == 0:
                self.state = np.array([0, 1, 0])
            else:
                self.state = np.array([0, 0, 1])
            global_rew = 0
            terminated = False
        elif state_index == 1:
            global_rew = 7
            terminated = True
        else:
            if action_dict[self.agent_1] == 0 and action_dict[self.agent_2] == 0:
                global_rew = 0
            elif action_dict[self.agent_1] == 1 and action_dict[self.agent_2] == 1:
                global_rew = 8
            else:
                global_rew = 1
            terminated = True
        

        rewards = {self.agent_1: global_rew / 2.0, self.agent_2: global_rew / 2.0}
        obs = self._obs()
        terminateds = {"__all__": terminated}
        truncateds = {"__all__": False}
        infos = {
            self.agent_1: {"done": terminateds["__all__"]},
            self.agent_2: {"done": terminateds["__all__"]},
        }
        return obs, rewards, terminateds, truncateds, infos

    def _obs(self):
        if self.with_state:
            return {
                self.agent_1: {"obs": self.agent_1_obs(), ENV_STATE: self.state},
                self.agent_2: {"obs": self.agent_2_obs(), ENV_STATE: self.state},
            }
        else:
            return {self.agent_1: self.agent_1_obs(), self.agent_2: self.agent_2_obs()}

    def agent_1_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [1]])
        else:
            return np.flatnonzero(self.state)[0]

    def agent_2_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [2]])
        else:
            return np.flatnonzero(self.state)[0] + 3
    def render():
        X = np.random.random((5, 2))
        Y = np.random.random((5, 2))
        gui = ti.GUI("lines", res=(400, 400))
        while gui.running:
            gui.lines(begin=X, end=Y, radius=2, color=0x068587)
            gui.show()
        #if self.render_mode == "rgb_array":
            #return self._render_frame()

"""
class TwoStepGameWithGroupedAgents(MultiAgentEnv):
    def __init__(self, env_config):
        super().__init__()
        env = TwoStepGame(env_config)
        tuple_obs_space = Tuple([env.observation_space, env.observation_space])
        tuple_act_space = Tuple([env.action_space, env.action_space])

        self.env = env.with_agent_groups(
            groups={"agents": [0, 1]},
            obs_space=tuple_obs_space,
            act_space=tuple_act_space,
        )
        self.observation_space = self.env.observation_space
        self.action_space = self.env.action_space
        self._agent_ids = {"agents"}

    def reset(self, *, seed=None, options=None):
        return self.env.reset(seed=seed, options=options)

    def step(self, actions):
        return self.env.step(actions)
"""

'\nclass TwoStepGameWithGroupedAgents(MultiAgentEnv):\n    def __init__(self, env_config):\n        super().__init__()\n        env = TwoStepGame(env_config)\n        tuple_obs_space = Tuple([env.observation_space, env.observation_space])\n        tuple_act_space = Tuple([env.action_space, env.action_space])\n\n        self.env = env.with_agent_groups(\n            groups={"agents": [0, 1]},\n            obs_space=tuple_obs_space,\n            act_space=tuple_act_space,\n        )\n        self.observation_space = self.env.observation_space\n        self.action_space = self.env.action_space\n        self._agent_ids = {"agents"}\n\n    def reset(self, *, seed=None, options=None):\n        return self.env.reset(seed=seed, options=options)\n\n    def step(self, actions):\n        return self.env.step(actions)\n'

In [ ]:
env = gym.make('InvertedPendulum-v4')

#act of -3,3
#obs of -inf, inf

In [ ]:
design = torch.tensor(torch.tensor([[1.], [-1.]]))#policies in the end can only do two things

In [ ]:
from torch import nn

class OutcomePredictor(nn.Module):

    def __init__(self):
        super().__init__()
        self.h1 = nn.Linear(51, 64)
        self.h2 = nn.Linear(64, 64)
        self.h3 = nn.Linear(64, 1)

    def compute_dem_probability(self, y):
        z = nn.functional.relu(self.h1(y))
        z = nn.functional.relu(self.h2(z))
        return self.h3(z)

    def forward(self, y_dict, design, observation_labels, target_labels):

        pyro.module("posterior_guide", self)

        y = y_dict["y"]
        dem_prob = self.compute_dem_probability(y).squeeze()
        pyro.sample("w", dist.Bernoulli(logits=dem_prob))

In [ ]:
prior_entropy = dist.Bernoulli(prior_w_prob).entropy()

In [ ]:
from gymnasium.spaces import Discrete, MultiDiscrete, Space
from typing import Union, Optional

from ray.rllib.utils.annotations import PublicAPI
from ray.rllib.models.action_dist import ActionDistribution
from ray.rllib.models.tf.tf_action_dist import Categorical
from ray.rllib.models.torch.torch_action_dist import TorchCategorical
from ray.rllib.utils.annotations import override
from ray.rllib.utils.exploration.stochastic_sampling import StochasticSampling
from ray.rllib.utils.framework import TensorType

@PublicAPI
class EFE(StochasticSampling):
    """Special case of StochasticSampling w/ Categorical and temperature param.
    Returns a stochastic sample from a Categorical parameterized by the model
    output divided by the temperature. Returns the argmax iff explore=False.
    """

    def __init__(
        self,
        action_space: Space,
        *,
        framework: Optional[str],
        temperature: float = 1.0,
        **kwargs
    ):
        """Initializes a SoftQ Exploration object.
        Args:
            action_space: The gym action space used by the environment.
            temperature: The temperature to divide model outputs by
                before creating the Categorical distribution to sample from.
            framework: One of None, "tf", "torch".
        """
        assert isinstance(action_space, (Discrete, MultiDiscrete))
        super().__init__(action_space, framework=framework, **kwargs)
        self.temperature = temperature

    @override(StochasticSampling)
    def get_exploration_action(
        self,
        action_distribution: ActionDistribution,
        timestep: Union[int, TensorType],
        explore: bool = True,
    ):
        cls = type(action_distribution)
        assert issubclass(cls, (Categorical, TorchCategorical))
        # Re-create the action distribution with the correct temperature
        # applied.
        dist = cls(action_distribution.inputs, self.model, temperature=self.temperature)
        #self.model  is the first agent model we use to get the first q value used for model selection
        # Delegate to super method.
        return super().get_exploration_action(
            action_distribution=dist, timestep=timestep, explore=explore

In [24]:
#our custom replacement for softq in explore config
from gymnasium.spaces import Discrete, MultiDiscrete, Space
from typing import Union, Optional

from ray.rllib.utils.annotations import PublicAPI
from ray.rllib.models.action_dist import ActionDistribution
from ray.rllib.models.tf.tf_action_dist import Categorical
from ray.rllib.models.torch.torch_action_dist import TorchCategorical
from ray.rllib.utils.annotations import override
from ray.rllib.utils.exploration.stochastic_sampling import StochasticSampling
from ray.rllib.utils.framework import TensorType

@PublicAPI
class SoftQ(StochasticSampling):
    """Special case of StochasticSampling w/ Categorical and temperature param.
    Returns a stochastic sample from a Categorical parameterized by the model
    output divided by the temperature. Returns the argmax iff explore=False.
    """

    def __init__(
        self,
        action_space: Space,
        *,
        framework: Optional[str],
        temperature: float = 1.0,
        **kwargs
    ):
        """Initializes a SoftQ Exploration object.
        Args:
            action_space: The gym action space used by the environment.
            temperature: The temperature to divide model outputs by
                before creating the Categorical distribution to sample from.
            framework: One of None, "tf", "torch".
        """
        assert isinstance(action_space, (Discrete, MultiDiscrete))
        super().__init__(action_space, framework=framework, **kwargs)
        self.temperature = temperature

    @override(StochasticSampling)
    def get_exploration_action(
        self,
        action_distribution: ActionDistribution,
        timestep: Union[int, TensorType],
        explore: bool = True,
    ):
        cls = type(action_distribution)
        assert issubclass(cls, (Categorical, TorchCategorical))
        # Re-create the action distribution with the correct temperature
        # applied.
        dist = cls(action_distribution.inputs, self.model, temperature=self.temperature)
        # Delegate to super method.
        return super().get_exploration_action(
            action_distribution=dist, timestep=timestep, explore=explore
        )

In [25]:
from typing import Optional, Type

from ray.rllib.algorithms.algorithm_config import AlgorithmConfig, NotProvided
from ray.rllib.algorithms.simple_q.simple_q import SimpleQ, SimpleQConfig
from ray.rllib.algorithms.qmix.qmix_policy import QMixTorchPolicy
from ray.rllib.utils.replay_buffers.utils import update_priorities_in_replay_buffer
from ray.rllib.execution.rollout_ops import (
    synchronous_parallel_sample,
)
from ray.rllib.execution.train_ops import (
    multi_gpu_train_one_step,
    train_one_step,
)
from ray.rllib.policy.policy import Policy
from ray.rllib.utils.annotations import override
from ray.rllib.utils.deprecation import Deprecated
from ray.rllib.utils.metrics import (
    LAST_TARGET_UPDATE_TS,
    NUM_AGENT_STEPS_SAMPLED,
    NUM_ENV_STEPS_SAMPLED,
    NUM_TARGET_UPDATES,
    SYNCH_WORKER_WEIGHTS_TIMER,
    SAMPLE_TIMER,
)
from ray.rllib.utils.replay_buffers.utils import sample_min_n_steps_from_buffer
from ray.rllib.utils.typing import ResultDict
from ray.rllib.utils.deprecation import DEPRECATED_VALUE
from ray.rllib.utils.deprecation import deprecation_warning
#@ti.kernel
class QMixConfig(SimpleQConfig):
    """Defines a configuration class from which QMix can be built.
    Example:
        >>> from ray.rllib.examples.env.two_step_game import TwoStepGame
        >>> from ray.rllib.algorithms.qmix import QMixConfig
        >>> config = QMixConfig()  # doctest: +SKIP
        >>> config = config.training(gamma=0.9, lr=0.01, kl_coeff=0.3)  # doctest: +SKIP
        >>> config = config.resources(num_gpus=0)  # doctest: +SKIP
        >>> config = config.rollouts(num_rollout_workers=4)  # doctest: +SKIP
        >>> print(config.to_dict())  # doctest: +SKIP
        >>> # Build an Algorithm object from the config and run 1 training iteration.
        >>> algo = config.build(env=TwoStepGame)  # doctest: +SKIP
        >>> algo.train()  # doctest: +SKIP
    Example:
        >>> from ray.rllib.examples.env.two_step_game import TwoStepGame
        >>> from ray.rllib.algorithms.qmix import QMixConfig
        >>> from ray import air
        >>> from ray import tune
        >>> config = QMixConfig()
        >>> # Print out some default values.
        >>> print(config.optim_alpha)  # doctest: +SKIP
        >>> # Update the config object.
        >>> config.training(  # doctest: +SKIP
        ...     lr=tune.grid_search([0.001, 0.0001]), optim_alpha=0.97
        ... )
        >>> # Set the config object's env.
        >>> config.environment(env=TwoStepGame)  # doctest: +SKIP
        >>> # Use to_dict() to get the old-style python config dict
        >>> # when running with tune.
        >>> tune.Tuner(  # doctest: +SKIP
        ...     "QMix",
        ...     run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
        ...     param_space=config.to_dict(),
        ... ).fit()
    """

    def __init__(self):
        """Initializes a PPOConfig instance."""
        super().__init__(algo_class=QMix)

        # fmt: off
        # __sphinx_doc_begin__
        # QMix specific settings:
        self.mixer = "qmix"
        self.mixing_embed_dim = 32
        self.double_q = True
        self.optim_alpha = 0.99
        self.optim_eps = 0.00001
        self.grad_clip = 10
        #self.render_mode = 'rgb_array'
        # QMix-torch overrides the TorchPolicy's learn_on_batch w/o specifying a
        # alternative `learn_on_loaded_batch` alternative for the GPU.
        # TODO: This hack will be resolved once we move all algorithms to the new
        #  RLModule/Learner APIs.
        self.simple_optimizer = True

        # Override some of AlgorithmConfig's default values with QMix-specific values.
        # .training()
        self.lr = 0.0005
        self.train_batch_size = 32
        self.target_network_update_freq = 500
        self.num_steps_sampled_before_learning_starts = 1000
        self.replay_buffer_config = {
            "type": "ReplayBuffer",
            # Specify prioritized replay by supplying a buffer type that supports
            # prioritization, for example: MultiAgentPrioritizedReplayBuffer.
            "prioritized_replay": DEPRECATED_VALUE,
            # Size of the replay buffer in batches (not timesteps!).
            "capacity": 1000,
            # Choosing `fragments` here makes it so that the buffer stores entire
            # batches, instead of sequences, episodes or timesteps.
            "storage_unit": "fragments",
            # Whether to compute priorities on workers.
            "worker_side_prioritization": False,
        }
        """
        self.model = {
            "lstm_cell_size": 64,
            "max_seq_len": 999999,
        }
        """
        # .framework()
        self.framework_str = "torch"

        # .rollouts()
        self.rollout_fragment_length = 4
        self.batch_mode = "complete_episodes"

        # .reporting()
        self.min_time_s_per_iteration = 1
        self.min_sample_timesteps_per_iteration = 1000

        # .exploration()
        self.exploration_config = {
            """
            # The Exploration class to use.
            "type": "EpsilonGreedy", #replace this with SoftQ
            # Config for the Exploration class' constructor:
            "initial_epsilon": 1.0,
            "final_epsilon": 0.01,
            # Timesteps over which to anneal epsilon.
            "epsilon_timesteps": 40000,
            """
            "type": "SoftQ",
            "temperature": 1.0
            # For soft_q, use:
            # "exploration_config" = {
            #   "type": "SoftQ"
            #   "temperature": [float, e.g. 1.0]
            # }
        }

        # .evaluation()
        # Evaluate with epsilon=0 every `evaluation_interval` training iterations.
        # The evaluation stats will be reported under the "evaluation" metric key.
        self.evaluation(
            evaluation_config=AlgorithmConfig.overrides(explore=False)
        )
        # __sphinx_doc_end__
        # fmt: on

        self.worker_side_prioritization = DEPRECATED_VALUE

    @override(SimpleQConfig)
    def training(
        self,
        *,
        mixer: Optional[str] = NotProvided,
        mixing_embed_dim: Optional[int] = NotProvided,
        double_q: Optional[bool] = NotProvided,
        target_network_update_freq: Optional[int] = NotProvided,
        replay_buffer_config: Optional[dict] = NotProvided,
        optim_alpha: Optional[float] = NotProvided,
        optim_eps: Optional[float] = NotProvided,
        grad_clip: Optional[float] = NotProvided,
        # Deprecated args.
        grad_norm_clipping=DEPRECATED_VALUE,
        **kwargs,
    ) -> "QMixConfig":
        """Sets the training related configuration.
        Args:
            mixer: Mixing network. Either "qmix", "vdn", or None.
            mixing_embed_dim: Size of the mixing network embedding.
            double_q: Whether to use Double_Q learning.
            target_network_update_freq: Update the target network every
                `target_network_update_freq` sample steps.
            replay_buffer_config:
            optim_alpha: RMSProp alpha.
            optim_eps: RMSProp epsilon.
            grad_clip: If not None, clip gradients during optimization at
                this value.
            grad_norm_clipping: Depcrecated in favor of grad_clip
        Returns:
            This updated AlgorithmConfig object.
        """
        # Pass kwargs onto super's `training()` method.
        super().training(**kwargs)

        if grad_norm_clipping != DEPRECATED_VALUE:
            deprecation_warning(
                old="grad_norm_clipping",
                new="grad_clip",
                help="Parameter `grad_norm_clipping` has been "
                "deprecated in favor of grad_clip in QMix. "
                "This is now the same parameter as in other "
                "algorithms. `grad_clip` will be overwritten by "
                "`grad_norm_clipping={}`".format(grad_norm_clipping),
                error=True,
            )
            grad_clip = grad_norm_clipping

        if mixer is not NotProvided:
            self.mixer = mixer
        if mixing_embed_dim is not NotProvided:
            self.mixing_embed_dim = mixing_embed_dim
        if double_q is not NotProvided:
            self.double_q = double_q
        if target_network_update_freq is not NotProvided:
            self.target_network_update_freq = target_network_update_freq
        if replay_buffer_config is not NotProvided:
            self.replay_buffer_config = replay_buffer_config
        if optim_alpha is not NotProvided:
            self.optim_alpha = optim_alpha
        if optim_eps is not NotProvided:
            self.optim_eps = optim_eps
        if grad_clip is not NotProvided:
            self.grad_clip = grad_clip

        return self

    @override(SimpleQConfig)
    def validate(self) -> None:
        # Call super's validation method.
        super().validate()

        if self.framework_str != "torch":
            raise ValueError(
                "Only `config.framework('torch')` supported so far for QMix!"
            )
#@ti.kernel
class QMix(SimpleQ):
    @classmethod
    @override(SimpleQ)
    def get_default_config(cls) -> AlgorithmConfig:
        return QMixConfig()

    @classmethod
    @override(SimpleQ)
    def get_default_policy_class(
        cls, config: AlgorithmConfig
    ) -> Optional[Type[Policy]]:
        return QMixTorchPolicy

    @override(SimpleQ)
    def training_step(self) -> ResultDict:
        """QMIX training iteration function.
        - Sample n MultiAgentBatches from n workers synchronously.
        - Store new samples in the replay buffer.
        - Sample one training MultiAgentBatch from the replay buffer.
        - Learn on the training batch.
        - Update the target network every `target_network_update_freq` sample steps.
        - Return all collected training metrics for the iteration.
        Returns:
            The results dict from executing the training iteration.
        """
        # Sample n batches from n workers.
        with self._timers[SAMPLE_TIMER]:
            new_sample_batches = synchronous_parallel_sample(
                worker_set=self.workers, concat=False
            )

        for batch in new_sample_batches:
            # Update counters.
            self._counters[NUM_ENV_STEPS_SAMPLED] += batch.env_steps()
            self._counters[NUM_AGENT_STEPS_SAMPLED] += batch.agent_steps()
            # Store new samples in the replay buffer.
            self.local_replay_buffer.add(batch)

        # Update target network every `target_network_update_freq` sample steps.
        cur_ts = self._counters[
            NUM_AGENT_STEPS_SAMPLED
            if self.config.count_steps_by == "agent_steps"
            else NUM_ENV_STEPS_SAMPLED
        ]

        train_results = {}

        if cur_ts > self.config.num_steps_sampled_before_learning_starts:
            # Sample n batches from replay buffer until the total number of timesteps
            # reaches `train_batch_size`.
            train_batch = sample_min_n_steps_from_buffer(
                replay_buffer=self.local_replay_buffer,
                min_steps=self.config.train_batch_size,
                count_by_agent_steps=self.config.count_steps_by == "agent_steps",
            )

            # Learn on the training batch.
            # Use simple optimizer (only for multi-agent or tf-eager; all other
            # cases should use the multi-GPU optimizer, even if only using 1 GPU)
            if self.config.get("simple_optimizer") is True:
                train_results = train_one_step(self, train_batch)
            else:
                train_results = multi_gpu_train_one_step(self, train_batch)

            # Update target network every `target_network_update_freq` sample steps.
            last_update = self._counters[LAST_TARGET_UPDATE_TS]
            if cur_ts - last_update >= self.config.target_network_update_freq:
                to_update = self.workers.local_worker().get_policies_to_train()
                self.workers.local_worker().foreach_policy_to_train(
                    lambda p, pid: pid in to_update and p.update_target()
                )
                self._counters[NUM_TARGET_UPDATES] += 1
                self._counters[LAST_TARGET_UPDATE_TS] = cur_ts

            update_priorities_in_replay_buffer(
                self.local_replay_buffer, self.config, train_batch, train_results
            )

            # Update weights and global_vars - after learning on the local worker -
            # on all remote workers.
            global_vars = {
                "timestep": self._counters[NUM_ENV_STEPS_SAMPLED],
            }
            # Update remote workers' weights and global vars after learning on local
            # worker.
            with self._timers[SYNCH_WORKER_WEIGHTS_TIMER]:
                self.workers.sync_weights(global_vars=global_vars)

        # Return all collected metrics for the iteration.
        return train_results


In [29]:
from gym.spaces import Dict, Discrete, Tuple, MultiDiscrete
#b = np.ones(3000).tolist()
#obs_space= Tuple({MultiDiscrete([])})

observation_space = Tuple(
    [
        Dict(
            {
                "obs": MultiDiscrete([3,3]), 
                ENV_STATE: MultiDiscrete([3,3]),
            }
        ),
        Dict(
            {
                "obs": MultiDiscrete([3,3]),
                ENV_STATE: MultiDiscrete([3,3]),
            }
        ),
    ]
)
action_space = Tuple(
    [
        Dict(
            {
                "obs": MultiDiscrete([1,3]), 
                ENV_STATE: MultiDiscrete([3,1]),
            }
        ),
        Dict(
            {
                "obs": MultiDiscrete([3,1]),
                ENV_STATE: MultiDiscrete([3,1]),
            }
        ),
    ]
)
"""
action_space = Tuple([
    "agent1": Dict(
        {
            #"prob": Discrete(100),
            "action_potential":  MultiDiscrete([1,1]), #excitatory neurons MultiDiscrete([2, 2, 2, 3])
            ENV_STATE: MultiDiscrete([2,2]),
        } ,dtype=np.float32),
    "agent2": Dict(
        {
            #"prob": Discrete(100),
            "action_potential": MultiDiscrete([1,2]), #inhibitory neurons MultiDiscrete([2, 2, 2, 3])
            ENV_STATE: MultiDiscrete([0,1]),
        } ,dtype=np.float32),
    "agent3": Dict(
        {
            #"prob": Discrete(100),
            "outsideAction":gym.spaces.Box(low=-1.0, high=1.0, shape=(10,)),#this should be replaced with the action space of inverted pendulum
            "action_potential": Discrete(1)# only excitatory
        }, dtype=np.float32)]
"""

'\naction_space = Tuple([\n    "agent1": Dict(\n        {\n            #"prob": Discrete(100),\n            "action_potential":  MultiDiscrete([1,1]), #excitatory neurons MultiDiscrete([2, 2, 2, 3])\n            ENV_STATE: MultiDiscrete([2,2]),\n        } ,dtype=np.float32),\n    "agent2": Dict(\n        {\n            #"prob": Discrete(100),\n            "action_potential": MultiDiscrete([1,2]), #inhibitory neurons MultiDiscrete([2, 2, 2, 3])\n            ENV_STATE: MultiDiscrete([0,1]),\n        } ,dtype=np.float32),\n    "agent3": Dict(\n        {\n            #"prob": Discrete(100),\n            "outsideAction":gym.spaces.Box(low=-1.0, high=1.0, shape=(10,)),#this should be replaced with the action space of inverted pendulum\n            "action_potential": Discrete(1)# only excitatory\n        }, dtype=np.float32)]\n'

In [15]:
ray.rllib.utils.tf_utils.get_gpu_devices()

[]

In [16]:
ray.shutdown()
#ray.init(num_cpus=16,num_gpus=1)

In [17]:
#policy spec 

In [36]:
#from gym.spaces import Dict, Discrete, Tuple, MultiDiscrete
from gymnasium.spaces import Dict, Discrete, MultiDiscrete, Tuple

logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument(
    "--run", type=str, default="PG", help="The RLlib-registered algorithm to use."
)
parser.add_argument(
    "--framework",
    choices=["tf", "tf2", "torch"],
    default="torch",
    help="The DL framework specifier.",
)
parser.add_argument("--num-cpus", type=int, default=16)
parser.add_argument("--num-gpus", type=int, default=1)
#parser.add_argument("render_mode", type=int, default=1)
parser.add_argument(
    "--mixer",
    type=str,
    default="qmix",
    choices=["qmix", "vdn", "none"],
    help="The mixer model to use.",
)
parser.add_argument(
    "--as-test",
    action="store_true",
    help="Whether this script should be run as a test: --stop-reward must "
    "be achieved within --stop-timesteps AND --stop-iters.",
)
parser.add_argument(
    "--stop-iters", type=int, default=200, help="Number of iterations to train."
)
parser.add_argument(
    "--stop-timesteps", type=int, default=70000, help="Number of timesteps to train."
)
parser.add_argument(
    "--stop-reward", type=float, default=8.0, help="Reward at which we stop training."
)
parser.add_argument(
    "--local-mode",
    action="store_true",
    help="Init Ray in local mode for easier debugging.",
)
if __name__ == "__main__":
    
    #def policy_mapping_fn(agent_id, episode, worker, **kwargs):
            #if agent_id.startswith("low_level_"):
                #return "low_level_policy"
            #else:
                #return "high_level_policy"
    args = parser.parse_args()

    ray.init(num_cpus=16, num_gpus=1,local_mode=args.local_mode)

    grouping = {
        "group_1": [0],
        "group_2": [0,1],
        "group_3": [0]
    }
    obs_space = Tuple(
        [
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 2]), 
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 2]),
                    ENV_STATE: MultiDiscrete([2, 2, 2,2]),
                }
            ),
        ]
    )
    #act_space =Tuple(
    #[
    #TwoStepGame.action_space,
    #TwoStepGame.action_space,
    #]
    #)
    act_space = Tuple(
        [
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]), 
                    ENV_STATE: MultiDiscrete([2, 2, 2,1]),
                }
            ),
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2,1]),
                }
            ),
        ]
    )
    register_env(
        "grouped_twostep",
        lambda config: TwoStepGame(config).with_agent_groups(
            grouping, obs_space=observation_space, act_space=action_space
        ),
    )
    """
    from ray.tune import register_env
    from ray.rllib.algorithms.dqn import DQN 
    YourExternalEnv = ... 
    register_env("my_env", 
        lambda config: YourExternalEnv(config))
    trainer = DQN(env="my_env") 
    while True: 
        print(trainer.train()) 

    
    """

    config = (
        get_trainable_cls(args.run)
        .get_default_config()
        .environment(TwoStepGame)
        .framework(args.framework)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )

    if args.run == "QMIX":
        (
            config.framework("torch")
            .training(mixer=args.mixer, train_batch_size=32)
            
            .multi_agent(
                policies={
                    "high_level_policy": (
                        None,
                        observation_space,
                        action_space,
                        config.overrides(gamma=0.9),
                    ),
                    "low_level_policy": (
                        None,
                        Tuple([observation_space]),#, Discrete(4)]),
                        action_space,
                        config.overrides(gamma=0.0),
                    ),
                },
                policy_mapping_fn=policy_mapping_fn,
            )
            .rollouts(num_rollout_workers=16, rollout_fragment_length=4)
            .exploration(
                exploration_config={
                    "final_epsilon": 0.0,
                }
            )
            .environment(
                env="grouped_twostep",
                env_config={
                    "separate_state_space": True,
                    "one_hot_state_encoding": True,
                },
            )
        )

    stop = {
        "episode_reward_mean": args.stop_reward,
        "timesteps_total": args.stop_timesteps,
        "training_iteration": args.stop_iters,
    }
    from ray.tune.schedulers import PopulationBasedTraining

    pbt_scheduler = PopulationBasedTraining(
        time_attr='training_iteration',
        metric='episode_reward_mean',#'loss',
        mode='min',
        perturbation_interval=1,
        hyperparam_mutations={
            "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
            "alpha": tune.uniform(0.0, 1.0),
        }
    )
    results = tune.Tuner(
        args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),
        
        tune_config=tune.TuneConfig(
            num_samples=4,
            scheduler=pbt_scheduler,
        ),
        
        param_space=config,
    ).fit()

    if args.as_test:
        check_learning_achieved(results, args.stop_reward)

    ray.shutdown()

2023-04-07 02:09:25,944	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(pid=24656) C:\Users\subar\anaconda3\envs\d\lib\site-packages\tensorflow_probability\python\__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=24656)   if (distutils.version.LooseVersion(tf.__version__) <


(PG pid=24656) [Taichi] version 1.5.0, llvm 15.0.1, commit 7b885c28, win, python 3.9.15


(PG pid=24656) [I 04/07/23 02:09:38.404 29300] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
(PG pid=24656) 2023-04-07 02:09:38,967	WARNING algorithm_config.py:596 -- Cannot create PGConfig from given `config_dict`! Property alpha not supported.
(PG pid=24656) 2023-04-07 02:09:39,204	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PG pid=24656) 2023-04-07 02:09:39,216	WARNING env.py:53 -- Skipping env checking for this experiment
(pid=6332) C:\Users\subar\anaconda3\envs\d\lib\site-packages\tensorflow_probability\python\__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=6332)   if (distutils.version.LooseVersion(tf.__version__) <


(PG pid=6332) [Taichi] version 1.5.0, llvm 15.0.1, commit 7b885c28, win, python 3.9.15


...

2023-04-07 02:14:50,919	INFO pbt.py:804 -- 

[PopulationBasedTraining] [Exploit] Cloning trial e7524_00003 (score = -7.000000) into trial e7524_00000 (score = -7.010000)

2023-04-07 02:14:50,920	INFO pbt.py:831 -- 

[PopulationBasedTraining] [Explore] Perturbed the hyperparameter config of triale7524_00000:
lr : 0.0005 --- (shift right) --> 0.0001
alpha : 0.5599171977768558 --- (resample) --> 0.04973318731905818

(PG pid=2980) [I 04/07/23 02:14:51.137 17576] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
(PG pid=2980) 2023-04-07 02:14:52,139	WARNING algorithm_config.py:596 -- Cannot create PGConfig from given `config_dict`! Property alpha not supported.
(PG pid=2980) 2023-04-07 02:14:52,414	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PG pid=2980) 2023-04-07 02:14:52,431	WARNING env.py:53 -- Skipping env checking for this experiment
(PG pid=2980) 2023-0

(PG pid=23024) [Taichi] version 1.5.0, llvm 15.0.1, commit 7b885c28, win, python 3.9.15


(PG pid=23024) [I 04/07/23 02:15:00.667 32660] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
(PG pid=23024) 2023-04-07 02:15:01,477	WARNING algorithm_config.py:596 -- Cannot create PGConfig from given `config_dict`! Property alpha not supported.
(PG pid=23024) 2023-04-07 02:15:01,739	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PG pid=23024) 2023-04-07 02:15:01,753	WARNING env.py:53 -- Skipping env checking for this experiment
(PG pid=23024) 2023-04-07 02:15:01,878	INFO trainable.py:791 -- Restored on 127.0.0.1 from checkpoint: C:\Users\subar\AppData\Local\Temp\checkpoint_tmp_6c9ead47a56f4d129aefa8a1f1785623
(PG pid=23024) 2023-04-07 02:15:01,878	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 196, '_timesteps_total': None, '_time_total': 72.00095629692078, '_episodes_total': 19600}
2023-04-07 02:15:02,359	INFO pbt.py:804 -- 

[P

(PG pid=26384) [Taichi] version 1.5.0, llvm 15.0.1, commit 7b885c28, win, python 3.9.15


(PG pid=26384) [I 04/07/23 02:15:10.377 15960] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
(PG pid=26384) 2023-04-07 02:15:10,992	WARNING algorithm_config.py:596 -- Cannot create PGConfig from given `config_dict`! Property alpha not supported.
(PG pid=26384) 2023-04-07 02:15:11,236	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PG pid=26384) 2023-04-07 02:15:11,250	WARNING env.py:53 -- Skipping env checking for this experiment
(PG pid=26384) 2023-04-07 02:15:11,351	INFO trainable.py:791 -- Restored on 127.0.0.1 from checkpoint: C:\Users\subar\AppData\Local\Temp\checkpoint_tmp_44654ee7cacc4797bb2f2130370d5c47
(PG pid=26384) 2023-04-07 02:15:11,351	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 197, '_timesteps_total': None, '_time_total': 72.20186853408813, '_episodes_total': 19700}
2023-04-07 02:15:12,834	INFO tune.py:798 -- Tot

In [ ]:
# Get the best result based on a particular metric.
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")

# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint

from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint(checkpoint_path)

In [ ]:
algo = config.build()

# Train.
while True:
    print(algo.train())

In [ ]:
policy = algo.get_policy()
# <ray.rllib.policy.eager_tf_policy.PPOTFPolicy_eager object at 0x7fd020165470>

# Run a forward pass to get model output logits. Note that complex observations
# must be preprocessed as in the above code block.
logits, _ = policy.model({"obs": np.array([[0.1, 0.2, 0.3, 0.4]])})
# (<tf.Tensor: id=1274, shape=(1, 2), dtype=float32, numpy=...>, [])

# Compute action distribution given logits
policy.dist_class
# <class_object 'ray.rllib.models.tf.tf_action_dist.Categorical'>
dist = policy.dist_class(logits, policy.model)
# <ray.rllib.models.tf.tf_action_dist.Categorical object at 0x7fd02301d710>

# Query the distribution for samples, sample logps
dist.sample()
# <tf.Tensor: id=661, shape=(1,), dtype=int64, numpy=..>
dist.logp([1])
# <tf.Tensor: id=1298, shape=(1,), dtype=float32, numpy=...>

# Get the estimated values for the most recent forward pass
policy.model.value_function()
# <tf.Tensor: id=670, shape=(1,), dtype=float32, numpy=...>

policy.model.base_model.summary()

In [ ]:
sess = Session(graph, env)
sess.run(
    n_steps=500000,
    learn=True,
    render=TwoStepGame.render,
    success_reward=0.0
)

In [ ]:
TwoStepGame.render()

In [ ]:
import numpy as np
pos = np.random.random((50, 2))
# Create an array of 50 integer elements whose values are randomly 0, 1, 2
# 0 corresponds to 0x068587
# 1 corresponds to 0xED553B
# 2 corresponds to 0xEEEEF0
indices = np.random.randint(0, 2, size=(50,))
gui = ti.GUI("circles", res=(400, 400))
while gui.running:
    gui.circles(pos, radius=5, palette=[0x068587, 0xED553B, 0xEEEEF0], palette_indices=indices)
    gui.show()

In [5]:
import taichi as ti

ti.init(arch=ti.cuda)

N = 10

particles_pos = ti.Vector.field(3, dtype=ti.f32, shape = N)
points_pos = ti.Vector.field(3, dtype=ti.f32, shape = N)

@ti.kernel
def init_points_pos(points : ti.template()):
    for i in range(points.shape[0]):
        points[i] = [i for j in ti.static(range(3))]

init_points_pos(particles_pos)
init_points_pos(points_pos)

window = ti.ui.Window("Test for Drawing 3d-lines", (768, 768))
canvas = window.get_canvas()
scene = ti.ui.Scene()
camera = ti.ui.Camera()
camera.position(5, 2, 2)

while window.running:
    camera.track_user_inputs(window, movement_speed=0.03, hold_key=ti.ui.RMB)
    scene.set_camera(camera)
    scene.ambient_light((0.8, 0.8, 0.8))
    scene.point_light(pos=(0.5, 1.5, 1.5), color=(1, 1, 1))

    scene.particles(particles_pos, color = (0.68, 0.26, 0.19), radius = 0.1)
    # Draw 3d-lines in the scene
    scene.lines(points_pos, color = (0.28, 0.68, 0.99), width = 5.0)
    canvas.scene(scene)
    window.show()

[Taichi] Starting on arch=cuda


In [4]:
import taichi as ti
import taichi.math as tm

ti.init(arch=ti.gpu)

n = 320
pixels = ti.field(dtype=float, shape=(n * 2, n))

@ti.func
def complex_sqr(z):  # complex square of a 2D vector
    return tm.vec2(z[0] * z[0] - z[1] * z[1], 2 * z[0] * z[1])

@ti.kernel
def paint(t: float):
    for i, j in pixels:  # Parallelized over all pixels
        c = tm.vec2(-0.8, tm.cos(t) * 0.2)
        z = tm.vec2(i / n - 1, j / n - 0.5) * 2
        iterations = 0
        while z.norm() < 20 and iterations < 50:
            z = complex_sqr(z) + c
            iterations += 1
        pixels[i, j] = 1 - iterations * 0.02

gui = ti.GUI("Julia Set", res=(n * 2, n))

for i in range(1000000):
    paint(i * 0.03)
    gui.set_image(pixels)
    gui.show()

[Taichi] version 1.4.1, llvm 15.0.1, commit e67c674e, win, python 3.9.15
[Taichi] Starting on arch=cuda


RuntimeError: Window close button clicked, exiting... (use `while gui.running` to exit gracefully)

In [ ]:
#from ray.rllib.examples.env.two_step_game import TwoStepGame

#it lacks the full paremterizan of above cell. That my be the cause of the error


config = QMixConfig()  
config = config.training(gamma=0.9, lr=0.01)#, kl_coeff=0.3)  
#config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=12)  
print(config.to_dict())  
# Build an Algorithm object from the config and run 1 training iteration.
register_env(
    "grouped_twostep",
    lambda config: TwoStepGame(config).with_agent_groups(
        grouping, obs_space=obs_space, act_space=act_space
    ),
)

algo = config.build(env="grouped_twostep")  
algo.train() 